In [1]:
from gpt2 import GPT2
import torch
from huggingface_hub import hf_hub_download
import pandas as pd

/home/rafael/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

shakespeare_text = './data/shakespeare/input.txt'

gpt = GPT2.build(
    dataset="shakespeare",
    data_path=shakespeare_text,
    model_size="gpt2",
    max_length=256,
    batch_size=2,
)

337920 this has a different length: 114, padding


### Data

In [2]:
REPO_ID = "nampdn-ai/tiny-strange-textbooks"
download = hf_hub_download(repo_id=REPO_ID, filename="data_part_0.parquet", repo_type="dataset", token=TOKEN)

In [3]:
gpt = GPT2.build(
    dataset="tinystrange",
    data_path=download,
    model_size="gpt2",
    max_length=256,
    batch_size=6,
)

Reading parquet file /home/rafael/.cache/huggingface/hub/datasets--nampdn-ai--tiny-strange-textbooks/snapshots/6066e9c6f9e75e18f3625a551087bd44fe8a84e0/data_part_0.parquet...
Tokenizing text...


In [5]:
inputs, labels = gpt.datamodule.dataset_train.__getitem__(0)

### Training

In [13]:
gpt.datamodule.dataset.df.head()

,text,tokens,len_tokens
0,ver 2008 R2 installed. I want to use Sync Fra...,"[3326, 3648, 371, 17, 6589, 13, 314, 765, 284,...",102
1,Title: Introduction to Birthday Invitations\n...,"[11851, 25, 22395, 284, 33511, 10001, 20597, 1...",232
2,# Lesson 1: Introduction to Suspense and Dram...,"[1303, 12892, 261, 352, 25, 22395, 284, 31922,...",229
3,all seasons. The results also showed that the...,"[477, 7028, 13, 383, 2482, 635, 3751, 326, 262...",92
4,Title: Introduction to Photinia Rubens - Brig...,"[11851, 25, 22395, 284, 5919, 43168, 6256, 641...",254


In [10]:
gpt.train(
    max_epochs=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-03-06 14:02:44.544135: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 14:02:44.755087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-06 14:02:46.053224: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
----------------------

Epoch 0:   0%|          | 19/80000 [00:12<14:27:14,  1.54it/s, v_num=3, train_loss=8.360]

/home/rafael/miniconda3/envs/gengar/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [10]:
print(gpt.generate(prompt="If you tickle us do we not laugh?", max_len=512))



Hence show, twice each other.

KING RICHARD II:
The worthy danger, my sovereign, I am wisThe senators of the market.
Nay,
Than'd.

QUEENRY VI:
Whilstmen, my lord?
Come,
Come thou be my own disgrace in arms once
Must are put Henry's utter it be heir,
The waste gave me leaveceund,
Whose.

The dost thou hast thou art thy dearchurch of thy lord,
KING RICHARD III:
The dost thou can afford
Father, and move,
O,
Than, my tongue.
Not your grace, and thy daughter gone to-kind both, in joy,
KING RICHARD II:
Ay, and he shall soon
God save their tender-believing and afterENCE:
As I were a bird with thy head.
Flatter my cheeks, my lord.

My thoughts, my kingdom!-- eye!

Come,

KING RICHARD II:
 shine heaven's face
QUEENRY BOLINGHAM:

WARWICK:
OKE:
Henry, doom of us twain a thousand hair it.
I have given the adultous sighs, in my mind hath sworn his lands.

Up as thou think on, mistrust;
JOHN OF YORK:
HENRY VI:
So do it out of Norfolk:
For once again.
Which thou saw him they shall breed thy hand wh

In [1]:
from gpt2.modules.data import TextDataset
import tiktoken

tokenizer = tokenizer = tiktoken.get_encoding("gpt2")
dataset = TextDataset.from_file("./data/shakespeare/input.txt", tokenizer)

337920 this has a different length: 114, padding


In [2]:
dataset.__getitem__(0)

(tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
          3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
           461,    11,  2740,    13,   198,   198,  5962, 22307,    25,   198,
          1639,   389,   477, 12939,  2138,   284,  4656,   621,   284,  1145,
           680,    30,   198,   198,  3237,    25,   198,  4965,  5634,    13,
         12939,    13,   198,   198,  5962, 22307,    25,   198,  5962,    11,
           345,   760,   327,  1872,   385,  1526, 28599,   318,  4039,  4472,
           284,   262,   661,    13,   198,   198,  3237,    25,   198,  1135,
           760,   470,    11,   356,   760,   470,    13,   198,   198,  5962,
         22307,    25,   198,  5756,   514,  1494,   683,    11,   290,   356,
          1183,   423, 11676,   379,   674,   898,  2756,    13,   198,  3792,
           470,   257, 15593,    30,   198,   198,  3237,    25,   198,  2949,
           517,  3375,   319,   470,    26,  1309,  

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer